In [1]:
import pandas as pd
import numpy as np


tourney_seeds = pd.read_csv("./Data/TourneySeeds.csv")
rank = []
teams = []
for seed, team in tourney_seeds.loc[:,['Seed','Team']].values:
    rank.append(int(seed[1:3]))
tourney_seeds["NumericalSeeds"] = rank

In [2]:
tourney_compact_results = pd.read_csv("./Data/TourneyCompactResults.csv")

In [17]:
tourney_seeds.head()

,Season,Seed,Team,NumericalSeeds
0,1985,W01,1207,1
1,1985,W02,1210,2
2,1985,W03,1228,3
3,1985,W04,1260,4
4,1985,W05,1374,5


In [18]:
result.head()

,Result,Season,Player,Opponent
0,1,1985,1116,1234
1,1,1985,1120,1345
2,1,1985,1207,1250
3,1,1985,1229,1425
4,1,1985,1242,1325


In [3]:
result = pd.DataFrame({
        "Season": pd.concat([tourney_compact_results['Season'], tourney_compact_results['Season']], ignore_index=True),
        "Player": pd.concat([tourney_compact_results['Wteam'], tourney_compact_results['Lteam']], ignore_index=True),
        "Opponent": pd.concat([tourney_compact_results['Lteam'], tourney_compact_results['Wteam']], ignore_index=True),
        "Result": np.hstack([np.ones(tourney_compact_results.shape[0], dtype=int), np.zeros(tourney_compact_results.shape[0], dtype=int)])
    }, columns=["Result", "Season", "Player", "Opponent"])

In [4]:
t = pd.merge(result, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t[['Result', 'Season','Player', 'Opponent', 'NumericalSeeds']]
t.columns=['Result', 'Season','Player', 'Opponent', 'PlayerSeed']
result = t

In [5]:
t = pd.merge(result, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t[['Result', 'Season', 'Player', 'Opponent', 'PlayerSeed', 'NumericalSeeds']]
t.columns = ['Result', 'Season','Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
result = t

In [23]:
regular = pd.read_csv("./Data/RegularSeasonDetailedResults.csv")
regular.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [10]:
regular_rev = regular.ix[:, [0,1,4,5,2,3,7,21,22,23,24,25,26,27,28,29,30,31,32,33,8,9,10,11,12,13,14,15,16,17,18,19,20]]
new_regular = pd.DataFrame(np.vstack([regular_rev.values, 
                                   regular.ix[:, [0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]].values], 
                                 ), columns = ["Season","Daynum","Player","PlayerScore","Opponent","OpponentScore","Numot","Wfgm","Wfga","Wfgm3","Wfga3","Wftm","Wfta","Wor","Wdr","Wast","Wto","Wstl","Wblk","Wpf","Lfgm","Lfga","Lfgm3","Lfga3","Lftm","Lfta","Lor","Ldr","Last","Lto","Lstl","Lblk","Lpf"])

In [12]:
loc = []
revloc = []
for itm in regular['Wloc'].values:
    if itm=='N':
        loc.append(0)
        revloc.append(0)
    elif itm=='H':
        loc.append(1)
        revloc.append(2)
    else:
        loc.append(2)
        revloc.append(1)
new_regular['PlayerLoc']=np.hstack([revloc, loc])
new_regular['PlayerResult']=np.hstack([np.zeros(regular.shape[0], dtype=np.int), np.ones(regular.shape[0], dtype=np.int)])

In [24]:
new_regular.head()

,Season,Daynum,Player,PlayerScore,Opponent,OpponentScore,Numot,Wfgm,Wfga,Wfgm3,...,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf,PlayerLoc,PlayerResult
0,2003,10,1328,62,1104,68,0,22,53,2,...,18,14,24,13,23,7,1,22,0,0
1,2003,10,1393,63,1272,70,0,24,67,6,...,19,15,28,16,13,4,4,18,0,0
2,2003,11,1437,61,1266,73,0,22,73,3,...,29,17,26,15,10,5,2,25,0,0
3,2003,11,1457,50,1296,56,0,18,49,6,...,31,6,19,11,12,14,2,18,0,0
4,2003,11,1208,71,1400,77,0,24,62,6,...,13,17,22,12,14,4,4,20,0,0


Effective field goal percentage

In [42]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfgm', 'Wfgm3', 'Wfga', 'Lfgm', 'Lfgm3', 'Lfga']].copy()
t["EfgO"] = np.require(t['Wfgm'].values, dtype=np.float)+np.require(t['Wfgm3'].values, dtype=np.float)/2
t["EfgD"] = np.require(t['Lfgm'].values, dtype=np.float)+np.require(t['Lfgm3'].values, dtype=np.float)/2
t = t.groupby(by=['Season', 'Player']).sum()
rst = pd.DataFrame({'EfgO': t['EfgO']/t['Wfga'], 'EfgD': t['EfgD']/t['Lfga']}, columns=['EfgO', 'EfgD'])
rst.head()

EfgO      EfgD
Season Player                    
2003   1102    0.579443  0.510522
       1103    0.534814  0.545809
       1104    0.475953  0.476190
       1105    0.457241  0.511089
       1106    0.479005  0.451505

In [43]:
result = pd.merge(result, rst, left_on=['Season', 'Player'], right_index=True)

,Result,Season,Player,Opponent,PlayerSeed,OpponentSeed,EfgO,EfgD
2272,1,2003,1421,1411,16,16,0.486339,0.518382
2273,0,2003,1421,1400,16,1,0.486339,0.518382
2274,0,2003,1163,1400,5,1,0.523580,0.429628
2300,1,2003,1163,1140,5,12,0.523580,0.429628
2301,1,2003,1163,1390,5,4,0.523580,0.429628


(1828, 7)

## Feature normalization

In [7]:
feature = result[['Result', 'Season', 'Player', 'Opponent']].copy()

In [8]:
#seeddelta
from scipy import stats


cdf = dict()
X = result['PlayerSeed']-result['OpponentSeed']
hist, bin_edges = np.histogram(X.values, bins=31, density=True)
hist /= hist.sum()
cdf['SeedDelta'] = stats.rv_discrete(name='SeedDelta', values=(bin_edges[:-1], hist))
feature['SeedDelta'] = cdf['SeedDelta'].cdf(X.values)

In [11]:
#playerseed
hist, bin_edges = np.histogram(result['PlayerSeed'].values, bins=16, density=True)
hist /= hist.sum()
cdf['PlayerSeed'] = stats.rv_discrete(name='PlayerSeed', values=(bin_edges[:-1], hist))
feature['PlayerSeed'] = cdf['PlayerSeed'].cdf(result['PlayerSeed'].values)

In [12]:
feature.head()

,Result,Season,Player,Opponent,SeedDelta,PlayerSeed
0,1,1985,1116,1234,0.596585,0.694878
1,0,1985,1116,1385,0.862683,0.694878
2,1,1985,1207,1385,0.514146,0.131220
3,0,1985,1246,1385,0.936341,0.847805
4,0,1985,1301,1385,0.607073,0.322439


Parameter tunning

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

seg = feature[(feature['Season']<2003) & (feature['Season'])<2013]
X = seg.ix[:, 4:6].values
Y = seg.ix[:, 0].values
param = {'C':[1000000, 100000, 10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001]}
lr = LogisticRegression()
cver = GridSearchCV(lr, param, scoring='log_loss', cv=5)
cver.fit(X, Y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [1000000, 100000, 10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [14]:
cver.best_params_

{'C': 10}

In [15]:
lr = LogisticRegression(C=10)
lr.fit(X, Y)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Test on test set

In [56]:
test = feature[(feature['Season']==2013)]
Xt = test.ix[:, 4:6].values
Yt = test.ix[:, 0].values
Yp = lr.predict_proba(Xt)
test['Pred']=Yp[:, 1]

/home/ike/.miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [95]:
Xt[2, :]

array([ 0.80878049,  0.64609756])

In [93]:
test

,Result,Season,Player,Opponent,SeedDelta,PlayerSeed,Pred
3564,1,2013,1299,1251,0.514146,1.000000,0.477876
3565,0,2013,1299,1257,1.000000,1.000000,0.121022
3566,0,2013,1161,1257,0.808780,0.646098,0.243410
3567,0,2013,1276,1257,0.672683,0.401707,0.370005
3568,0,2013,1332,1257,0.936341,0.847805,0.155747
3569,0,2013,1455,1257,0.862683,0.694878,0.204492
3570,0,2013,1181,1257,0.596585,0.236098,0.453331
3571,1,2013,1388,1292,0.514146,0.798780,0.492531
3572,0,2013,1388,1272,0.753902,0.798780,0.275951
3573,1,2013,1277,1272,0.392927,0.322439,0.641441


In [37]:
Yp

array([[ 0.52212361,  0.47787639],
       [ 0.87897794,  0.12102206],
       [ 0.65793354,  0.34206646],
       [ 0.79550755,  0.20449245],
       [ 0.70833871,  0.29166129],
       [ 0.81967106,  0.18032894],
       [ 0.54666893,  0.45333107],
       [ 0.50746933,  0.49253067],
       [ 0.72404884,  0.27595116],
       [ 0.80039193,  0.19960807],
       [ 0.43917832,  0.56082168],
       [ 0.84717593,  0.15282407],
       [ 0.52212361,  0.47787639],
       [ 0.87897794,  0.12102206],
       [ 0.62999549,  0.37000451],
       [ 0.54666893,  0.45333107],
       [ 0.7565896 ,  0.2434104 ],
       [ 0.50746933,  0.49253067],
       [ 0.72404884,  0.27595116],
       [ 0.26975291,  0.73024709],
       [ 0.80619026,  0.19380974],
       [ 0.57227145,  0.42772855],
       [ 0.767257  ,  0.232743  ],
       [ 0.80266593,  0.19733407],
       [ 0.34583919,  0.65416081],
       [ 0.82567761,  0.17432239],
       [ 0.57956658,  0.42043342],
       [ 0.79550755,  0.20449245],
       [ 0.62999549,

In [34]:
#keeps the best auc
from sklearn.metrics import log_loss

log_loss(Yt, Yp)

0.52649896905457294

In [38]:
from sklearn.metrics import log_loss
log_loss(Yt, Yp)

0.59969320347157529

In [35]:
#keeps the best auc
from sklearn.metrics import roc_auc_score

roc_auc_score(Yt, Yp[:,1])

0.82223212296725323

In [39]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Yt, Yp[:,1])

0.74504343951882379

# Submit result

In [69]:
submission = pd.read_csv("./Data/sample_submission.csv")
content = []
for row in submission.iterrows():
    content.append(map(int, row[1].ix[0].split('_')))

In [70]:
submission = pd.DataFrame(content, columns=['Season', 'Player', 'Opponent'])

In [79]:
# take features from existing data
t = pd.merge(submission, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t.ix[:, [0,1,2,5]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed']
t = pd.merge(t, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t.ix[:, [0, 1, 2, 3, 6]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
t['SeedDelta'] = cdf['SeedDelta'].cdf(t['PlayerSeed']-t['OpponentSeed'])
t['PlayerSeed'] = cdf['PlayerSeed'].cdf(t['PlayerSeed'])

In [100]:
t[(t['Player']==1161) & (t['Opponent']==1257)]

,Season,Player,Opponent,PlayerSeed,OpponentSeed,SeedDelta
416,2013,1161,1257,0.646098,1,0.80878


In [48]:
X = t.ix[:, [5, 3]].values
Y = lr.predict_proba(X)

In [111]:
t['Prediction']=Y[:, 1]

In [115]:
id = []
pred = []
for row in t.iterrows():
    id.append("{0}_{1}_{2}".format(int(row[1].ix[0]), int(row[1].ix[1]), int(row[1].ix[2])))
    pred.append(row[1].ix[6])

In [116]:
submit = pd.DataFrame({"id":id, "pred":pred}, columns=['id', 'pred'])

In [117]:
submit.head()

,id,pred
0,2013_1103_1107,0.605500
1,2013_1103_1112,0.270094
2,2013_1107_1112,0.171374
3,2013_1103_1125,0.409608
4,2013_1107_1125,0.310642


In [125]:
submit.to_csv("result.csv", index=False)

Try to calculate loss locally

In [119]:
submission = t[['Season', 'Player', 'Opponent', 'Prediction']]

In [110]:
submission.shape

(9112, 4)

In [120]:
submission[(submission['Player']==1161) & (submission['Opponent']==1257)]

,Season,Player,Opponent,Prediction
416,2013,1161,1257,0.24341


In [121]:
t1 = pd.merge(submission, tourney_compact_results, left_on=['Season', 'Player', 'Opponent'], right_on=['Season', 'Wteam', 'Lteam'])
t2 = pd.merge(submission, tourney_compact_results, left_on=['Season', 'Player', 'Opponent'], right_on=['Season', 'Lteam', 'Wteam'])

In [124]:
t1['Label']=1
t2['Label']=0
submission = pd.concat([t1, t2], ignore_index=True)
log_loss(submission['Label'].values, submission['Prediction'].values)

0.59209624341275435

In [246]:
submission['pred'] = Y[:,0]

In [247]:
submission.to_csv("result.csv", index=False)